# 1-1　基本構成

在程式碼資料夾中有以下檔案：

- requirements.txt 裡面寫會用到那些 library
- app.py 主程式
- user.py 使用者相關類別
- database.py 和資料庫溝通的程式

在 user.py 的程式碼：

In [ ]:
import psycopg2

class User:
    def __init__(self, id, email, first_name, last_name):
        self.id = id
        self.email = email
        self.first_name = first_name
        self.last_name = last_name

    def __repr__(self):
        return "User {} {}".format(self.first_name, self.email)

`__repr__` 函式回傳字串，之後 `print` 物件時會顯示此字串

`__repr__` 為程式開發者導向，而類似功能的 `__str__` 為使用者導向、重視可讀性

In [ ]:
    def save_to_db(self):
        connection = psycopg2.connect\
        (user='postgres', password='1234', database ='learning', host='localhost')
        
        with connection.cursor() as cursor:
            cursor.execute\
            ('INSERT INTO users (email, first_name, last_name) VALUES (%s, %s, %s)',\
            (self.email, self.first_name, self.lastname))
            
        connection.commit()
        connection.close()

`connection` 建立完後，需要藉由 `cursor` 來存取資料

`with` 可以建立 `cursor` ，並在用完之後自動關閉

`cursor.execute()` 當中的 `%s` 為外卡字元，會傳入逗號後方的指定參數 

`psycopg2` 與資料庫的溝通流程：

- `psycopg2.connect()` 與資料庫建立連線
- `with connection.cursor()` 建立 `cursor`
- `cursor.execute(SQL)` 執行 SQL 語法
- (`cursor` 執行完後自動結束)
- `connection.commit()` 執行語法__並存檔__ (源於早期硬碟讀寫耗時，故為分離的指令)
- `connection.clost()` 關閉連線

# 1-2　基本構成 - 簡化

使用 `with` 再次精簡化：

In [ ]:
class User:
    def save_to_db(self):
        with psycopg2.connect(user='postgres', password='1234',\
                              database ='learning', host='localhost') as connection:
        
            with connection.cursor() as cursor:
                cursor.execute\
                ('INSERT INTO users (email, first_name, last_name) VALUES (%s, %s, %s)',\
                (self.email, self.first_name, self.lastname))

除了存入資料庫，我們還需要從資料庫搜尋並讀取資料：

In [ ]:
    @classmethod
    def load_from_db_by_email(cls, email):
        with psycopg2.connect(user='postgres', password='1234',\
                              database ='learning', host='localhost') as connection:
        with connection.cursor() as cursor:
                cursor.execute\
                ('SELECT * FROM users WHERE email=%s', (email,))
                user_data = cursor.fetchone()
                return cls(*user_data)

這裡使用 `classmethod` 因為查詢完後要回傳一個 `User` 類別的物件

而不是像一般函式傳入 `self` 對物件本身做操作

注意：`cursor.execute()` 的參數傳入需要是 tuple，單一元素傳遞下需寫成 `(email,)`

`cursor.fetchone()` 會回傳第一筆存在 `cursor` 中的搜尋結果

`*user_data` 即 `user_data[0], user_data[1], user_data[2], user_data[3]`  
(對應到 `id, email, first_name, last_name`)

另外，注意到重複的程式碼，我們可以把 `connect` 放到 database.py：

In [ ]:
import psycopg2

def connect():
    return psycopg2.connect(user='postgres', password='1234',\
                              database ='learning', host='localhost')

如此在 user.py 就不用引入 `psycopg2` ，並簡化為：

In [ ]:
from database import connect

class User:
    def save_to_db(self):
        with connect() as connection:            

***

# 2-1　Connection Pool

在上述的程式當中，每次存取資料庫都會包含連接與斷開連接

大量執行這些函式會進行多次的連線與中斷連線，造成伺服器負擔

我們可以使用 Connection Pool 來維持連線容量，不用每次執行函式時都進行連接或中斷

在 database.py 中：

In [ ]:
from psycopg2 import pool

connection_pool = pool.SimpleConnectionPool(1, 10,\
                                            user='postgres', password='1234',\
                                            database ='learning', host='localhost')

`pool.SimpleConnectionPool(最低維持連接數, 最高連接數, 連接資訊)`

如此在 user.py 可改為：

In [ ]:
from database import connection_pool

class User:
    def save_to_db(self):
        with connection_pool.getconn() as connection:  

`pool.getconn()` 建立連接到連線池

要注意的是，Connection Pool 本身並沒有針對 `with` 寫定機制

亦即就算寫了 `with connection_pool.getconn() as connection:`

它的作用只是 `connection = connection_pool.getconn()` 而並不會在關閉時歸還連線

故若最大連線數改為 1，再進行寫入及讀取會跳出 連線數用盡 的錯誤，因為寫入動作結束未歸還

我們可以暫時修改為：


In [ ]:
from database import connection_pool

class User:
    def save_to_db(self):
        connection = connection_pool.getconn()
        with connection.cursor() as cursor:
            cursor.execute('...')
        connection_pool.putconn(connection)
    
    @classmethod
    def load_from_db_by_email(cls, email):
        connection = connection_pool.getconn()
        with connection.cursor() as cursor:
            cursor.execute('...')
            return cls(*user_data)

使用 `connection_pool.putconn(connection)` 來歸還連線位置

但是另一個問題產生：

在 `load_from_db_by_email()` 最後會 `return` 回傳物件，無法直接歸還連線

故我們還是希望能使用 `with ConnectionPool() as connection:` 的語法

將 `return` 包在其下層的 `cursor`，最後關閉 `connection` 時可以歸還連線

因此我們會在 database.py 當中建立一個 `ConnectionPool` 類別來定義 `with` 對應的操作：

In [ ]:
from psycopg2 import pool

class ConnectionPool:
    
    def __init__(self):
        connection_pool = pool.SimpleConnectionPool(1, 10,\
                                            user='postgres', password='1234',\
                                            database ='learning', host='localhost')

    def __enter__(self):
        return self.connection_pool.get_conn()

    def __exit__(self, exception_type, exception_value, exception_traceback):
        pass

如果是 `with` 呼叫函式，會執行 `__init__` 之後接續執行 `__enter__`

不過這樣還沒有寫到歸還連線的機制，而且每次存取 `ConnectionPool` 都會創造一個新連線池

造成每次動作都會創造新的連線池出來，問題更為嚴重

正確的寫法：

In [ ]:
from psycopg2 import pool

connection_pool = pool.SimpleConnectionPool(1, 10,\
                                            user='postgres', password='1234',\
                                            database ='learning', host='localhost')

class ConnectionFromPool:
    
    def __init__(self):
        self.connection = None
        
    def __enter__(self):
        self.connection = connection_pool.getconn()
        return self.connection

    def __exit__(self, exception_type, exception_value, exception_traceback):
        self.connection.commit()
        connection_pool.putconn(self.connection)

( 將 `ConnectionPool` 改名為 `ConnectionFromPool` )

如此，匯入 database.py 時就會創建 連線池

且在需要執行動作時，會再建立 連線 而非 連線池，最後也會歸還 連線 到連線池裡

注意要加入 `self.connection.commit()` ，因為函式是我們自定義的

# 2-2　Connection Pool - 優化

現在 user.py 裡的函式成為：

In [ ]:
    def save_to_db(self):
        with ConnectionFromPool() as connection:
            with connection.cursor() as cursor:

在 `load_from_db_by_email` 函式也可發現此重複的巢狀 `with` 結構

故我們可以把 `cursor` 的操作也併入，進一步改成 `CursorFromConnectionFromPool` 函式：

In [ ]:
class CursorFromConnectionFromPool:

    def __init__(self):
        self.connection = None
        self.cursor = None

    def __enter__(self):
        self.connection = Database.get_conn()
        self.cursor = self.connection.cursor()
        return self.cursor

    def __exit__(self, exception_type, exception_value, exception_traceback):
        self.cursor.close()
        self.connection.commit()
        connection_pool.putconn(self.connection)

如此在 user.py 當中的函式就可以簡化為：

In [ ]:
    def save_to_db(self):
        with CursorFromConnectionFromPool() as cursor:
            cursor.execute('...')

另外，每次存取 database.py 時都會立刻開啟一個連線池，會造成隱性資源效率問題

我們希望改成 有需要時 在開啟連線池，故可新增一個 `Database` 類別處理：

In [ ]:
class Database:
    __connection_pool = None

    @classmethod
    def initialize(cls, **kwargs):
        cls.__connection_pool = pool.SimpleConnectionPool(1, 10, **kwargs)   

`__connection_pool` 在前面加兩底線的意思是 隱藏參數 (private)

因為目前所有的連線都會屬於同一個資料庫，且只會有一個連線池，故使用 `classmethod`

若使用一般函式及 `self` 參數，每個不同的連線物件都會創造自己的連線池出來

在某些支援自動完成的程式編輯器當中，不會被提示出來，以避免意外的修改

如此在主程式 app.py 中指定參數才會建立連線池

In [ ]:
Database.initialize(database="Learning", host="localhost",\
                    user="postgres", password="1234")

我們也可以把連線相關方法放進 `Database` 類別當中集中管理：

In [ ]:
    @classmethod
    def get_conn(cls):
        return cls.__connection_pool.getconn()

    @classmethod
    def return_conn(cls, connection):
        return cls.__connection_pool.putconn(connection)

    @classmethod
    def close_all_conn(cls):
        return cls.__connection_pool.closeall()

# 2-3　錯誤處理

當 `with` 處理的內容發生錯誤時，可以在 `__exit__` 中進行處理 (若不處理，程式會中止)

In [ ]:
    def __exit__(self, exception_type, exception_value, exception_traceback):
        if exception_value is not None:
            self.connection.rollback()
        else:
            self.cursor.close()
            self.connection.commit()
        Database.return_conn(self.connection)

- `exception_type` 錯誤的種類，例如：TypeError、AttributeError、ValueError
- `exception_value` 錯誤碼
- `exception_traceback` 錯誤說明文字

`connection.rollback()` 會直接不存檔就把連線中止並放回連線池